<a href="https://colab.research.google.com/github/hammadkhann/Effective-Dense-Retrieval/blob/main/Neural_Reranking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Pyterrier**

In [2]:
!pip -q install python-terrier
import pyterrier as pt
pt.init()

     |████████████████████████████████| 95 kB 2.2 MB/s 
     |████████████████████████████████| 1.1 MB 39.9 MB/s 
     |████████████████████████████████| 69 kB 6.9 MB/s 
     |████████████████████████████████| 255 kB 47.4 MB/s 
     |████████████████████████████████| 294 kB 50.6 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 1.8 MB 34.6 MB/s 
     |████████████████████████████████| 72 kB 1.5 MB/s 
     |████████████████████████████████| 126 kB 50.5 MB/s 
     |████████████████████████████████| 6.3 MB 37.5 MB/s 
     |████████████████████████████████| 291 kB 37.6 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


## **Importing useful libraries**



In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pickle
from google.colab import drive
from pyterrier.measures import *

pd.options.display.max_rows = None
pd.options.display.max_columns = None

## **Connect Google Drive**


In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


# **Preparing testing dataset**

## **DPH(Sparse Retrieval)**

## TREC-2019

In [5]:
br = pt.BatchRetrieve.from_dataset('msmarco_passage', 'terrier_stemmed_text', num_results=100, metadata = ['docno', 'text'], verbose=True)

/usr/local/lib/python3.7/dist-packages/pyterrier/datasets.py:273: UserWarning: Downloading index of > 2GB.
  warn("Downloading index of > 2GB.")


data.direct.bf:   0%|          | 0.00/486M [00:00<?, ?iB/s]

data.document.fsarrayfile:   0%|          | 0.00/177M [00:00<?, ?iB/s]

data.inverted.bf:   0%|          | 0.00/377M [00:00<?, ?iB/s]

data.lexicon.fsomapfile:   0%|          | 0.00/100M [00:00<?, ?iB/s]

...

data.meta.idx:   0%|          | 0.00/67.5M [00:00<?, ?iB/s]

data.meta.zdata:   0%|          | 0.00/1.91G [00:00<?, ?iB/s]

data.properties:   0%|          | 0.00/4.33k [00:00<?, ?iB/s]

md5sums:   0%|          | 0.00/537 [00:00<?, ?iB/s]

17:35:04.841 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


In [6]:
dph_2019 = pd.DataFrame()
topics = pt.get_dataset('msmarco_passage').get_topics('test-2019')
topics = pt.get_dataset('msmarco_passage').get_qrels('test-2019')[["qid"]].drop_duplicates().merge(topics, on=['qid'])
for res in br.transform_gen(topics, batch_size=10):
  dph_2019 = dph_2019.append(res)
print("Number of topics: ", len(topics))

msmarco-test2019-queries.tsv.gz:   0%|          | 0.00/4.18k [00:00<?, ?iB/s]

2019qrels-docs.txt:   0%|          | 0.00/183k [00:00<?, ?iB/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/3 [00:00<?, ?q/s]

Number of topics:  43


In [7]:
dph_2019.shape

(4205, 7)

## TREC-2020

In [8]:
dph_2020 = pd.DataFrame()
topics = pt.get_dataset('msmarco_passage').get_topics('test-2020')
topics = pt.get_dataset('msmarco_passage').get_qrels('test-2020')[["qid"]].drop_duplicates().merge(topics, on=['qid'])
for res in br.transform_gen(topics, batch_size=10):
  dph_2020 = dph_2020.append(res)
  
print("Number of topics: ", len(topics))

msmarco-test2020-queries.tsv.gz:   0%|          | 0.00/4.03k [00:00<?, ?iB/s]

2020qrels-docs.txt:   0%|          | 0.00/213k [00:00<?, ?iB/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/10 [00:00<?, ?q/s]

BR(DPH):   0%|          | 0/4 [00:00<?, ?q/s]

Number of topics:  54


In [9]:
dph_2020.shape

(5329, 7)

## **ColBERT(Dense Retrieval)**

## TREC-2019

In [10]:
index = pt.IndexFactory.of(pt.get_dataset("msmarco_passage").get_index("terrier_stemmed_text"))
print(index.getCollectionStatistics())

17:35:18.222 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.
Number of documents: 8841823
Number of terms: 1170682
Number of postings: 215238456
Number of fields: 1
Number of tokens: 288759529
Field names: [text]
Positions:   false



In [11]:
colbert_2019 = pt.io.read_results("/content/drive/MyDrive/E2E.2019.res")
# get document text for each docid
meta = index.getMetaIndex()
text = []
for index, row in colbert_2019.iterrows():
    text.append(meta.getItem('text', int(row['docno'])))
colbert_2019['text'] = text

# get query for each qid
topics_2019 = pt.get_dataset('msmarco_passage').get_topics('test-2019')
colbert_2019 = pd.merge(colbert_2019, topics_2019, on='qid', how='left')

# filter queries on 43 topics
colbert_2019 = colbert_2019[colbert_2019['qid'].isin(dph_2019['qid'].unique())]
# filter data on top100 results for each 43 topics
colbert_2019 = colbert_2019.groupby('qid').apply(lambda x : x.sort_values(by = 'score', ascending = False).head(100).reset_index(drop = True))
# reset index
colbert_2019.reset_index(drop=True, inplace=True)
colbert_2019.shape

(4300, 7)

## TREC-2020

In [12]:
colbert_2020 = pt.io.read_results("/content/drive/MyDrive/E2E.2020.res")
index = pt.IndexFactory.of(pt.get_dataset("msmarco_passage").get_index("terrier_stemmed_text"))

# get document text for each docid
meta = index.getMetaIndex()
text = []
for index, row in colbert_2020.iterrows():
    text.append(meta.getItem('text', int(row['docno'])))
colbert_2020['text'] = text

# get query for each qid
topics_2020 = pt.get_dataset('msmarco_passage').get_topics('test-2020')
colbert_2020 = pd.merge(colbert_2020, topics_2020, on='qid', how='left')

# filter queries on 43 topics
colbert_2020 = colbert_2020[colbert_2020['qid'].isin(dph_2020['qid'].unique())]
# filter data on top100 results for each 43 topics
colbert_2020 = colbert_2020.groupby('qid').apply(lambda x : x.sort_values(by = 'score', ascending = False).head(100).reset_index(drop = True))
# reset index
colbert_2020.reset_index(drop=True, inplace=True)
colbert_2020.shape

17:36:26.000 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 1.9 GiB of memory would be required.


(5400, 7)

# **Install Pyterrier-ColBERT**

In [13]:
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_colbert.git

  Cloning https://github.com/terrierteam/pyterrier_colbert.git to /tmp/pip-req-build-zg6ngxtw
  Running command git clone -q https://github.com/terrierteam/pyterrier_colbert.git /tmp/pip-req-build-zg6ngxtw
  Cloning https://github.com/cmacdonald/ColBERT.git (to revision v0.2) to /tmp/pip-install-thb8d2r_/colbert_00057fb4e2064c30b4348da64c7bce2f
  Running command git clone -q https://github.com/cmacdonald/ColBERT.git /tmp/pip-install-thb8d2r_/colbert_00057fb4e2064c30b4348da64c7bce2f
  Running command git checkout -b v0.2 --track origin/v0.2
  Switched to a new branch 'v0.2'
  Branch 'v0.2' set up to track remote branch 'v0.2' from 'origin'.
     |████████████████████████████████| 769 kB 5.5 MB/s 
     |████████████████████████████████| 216 kB 40.8 MB/s 
     |████████████████████████████████| 15.5 MB 37.8 MB/s 
     |████████████████████████████████| 3.0 MB 43.0 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 1.2 MB 26.9 MB/s 
    

In [14]:
from pyterrier_colbert.ranking import ColBERTFactory

## Updating pyterrier-ColBERT load methods to support latest version of transformers library.

In [15]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/Luyu/condenser

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 43 not upgraded.
Need to get 6,526 kB of archives.
After this operation, 14.7 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 3.0.2 [6,526 kB]
Fetched 6,526 kB in 1s (11.3 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [16]:
import os
import ujson
import torch
import random

from collections import defaultdict, OrderedDict

from colbert.parameters import DEVICE
import colbert.utils.utils
from colbert.utils.utils import print_message


def _load_checkpoint(path, model, optimizer=None, do_print=True):
    if do_print:
        print_message("#> Loading checkpoint", path)

    checkpoint = torch.load(path, map_location='cpu')

    state_dict = checkpoint['model_state_dict']
    new_state_dict = OrderedDict()
    for k, v in state_dict.items():
        name = k
        if k[:7] == 'module.':
            name = k[7:]
        new_state_dict[name] = v

    checkpoint['model_state_dict'] = new_state_dict

    model.load_state_dict(checkpoint['model_state_dict'], strict=False)

    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'], strict=False)

    if do_print:
        print_message("#> checkpoint['epoch'] =", checkpoint['epoch'])
        print_message("#> checkpoint['batch'] =", checkpoint['batch'])

    return checkpoint

colbert.utils.utils.load_checkpoint = _load_checkpoint
colbert.utils.utils.load_checkpoint.__globals__['load_checkpoint'] = _load_checkpoint
from colbert.modeling.colbert import ColBERT

def _load_model(args, do_print=True):
    colbert = ColBERT.from_pretrained('/content/condenser/',
                                      query_maxlen=args.query_maxlen,
                                      doc_maxlen=args.doc_maxlen,
                                      dim=args.dim,
                                      similarity_metric=args.similarity,
                                      mask_punctuation=args.mask_punctuation)
    colbert = colbert.to(DEVICE)

    print_message("#> Loading model checkpoint.", condition=do_print)

    checkpoint = load_checkpoint(args.checkpoint, colbert, do_print=do_print)

    colbert.eval()

    return colbert, checkpoint

In [17]:
import colbert.evaluation.load_model
import pyterrier_colbert.ranking
colbert.evaluation.load_model.load_model = _load_model
pyterrier_colbert.ranking.load_model = _load_model

# **TREC 2019 Experiments**

## **Loading model checkpoints**

### Baseline

In [18]:
colbert_checkpoint="/content/drive/MyDrive/colbert-50000.dnn" 
colbert_factory = ColBERTFactory(colbert_checkpoint, None, None)
colbert_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  colbert_factory.text_scorer()
colbert_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  colbert_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 17:38:53] #> Loading model checkpoint.
[Dec 01, 17:38:53] #> Loading checkpoint /content/drive/MyDrive/colbert-50000.dnn
[Dec 01, 17:39:11] #> checkpoint['epoch'] = 0
[Dec 01, 17:39:11] #> checkpoint['batch'] = 50000


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

### RD1: Additional signal during ColBERT training

In [19]:
cls_similarity_checkpoint="/content/drive/MyDrive/colbert-increase-similarity.dnn" 
cls_similarity_factory = ColBERTFactory(cls_similarity_checkpoint, None, None)
cls_similarity_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  cls_similarity_factory.text_scorer()
cls_similarity_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  cls_similarity_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 17:39:17] #> Loading model checkpoint.
[Dec 01, 17:39:17] #> Loading checkpoint /content/drive/MyDrive/colbert-increase-similarity.dnn
[Dec 01, 17:39:39] #> checkpoint['epoch'] = 0
[Dec 01, 17:39:39] #> checkpoint['batch'] = 50000


In [20]:
token_overlap_checkpoint="/content/drive/MyDrive/colbert-token-overlap-50000.dnn" 
token_overlap_factory = ColBERTFactory(token_overlap_checkpoint, None, None)
token_overlap_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  token_overlap_factory.text_scorer()
token_overlap_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  token_overlap_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 17:39:43] #> Loading model checkpoint.
[Dec 01, 17:39:43] #> Loading checkpoint /content/drive/MyDrive/colbert-token-overlap-50000.dnn
[Dec 01, 17:40:18] #> checkpoint['epoch'] = 0
[Dec 01, 17:40:18] #> checkpoint['batch'] = 50000


### RD2: Changing Underlying Pre-trained Model 

In [21]:
coltacl_checkpoint="/content/drive/MyDrive/colTaCL-50000.dnn" 
coltacl_factory = ColBERTFactory(coltacl_checkpoint, None, None)
coltacl_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  coltacl_factory.text_scorer()
coltacl_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  coltacl_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 17:40:22] #> Loading model checkpoint.
[Dec 01, 17:40:22] #> Loading checkpoint /content/drive/MyDrive/colTaCL-50000.dnn
[Dec 01, 17:41:01] #> checkpoint['epoch'] = 0
[Dec 01, 17:41:01] #> checkpoint['batch'] = 50000


In [22]:
colCondenser_checkpoint="/content/drive/MyDrive/colCondenser-50000.dnn" 
colCondenser_factory = ColBERTFactory(colCondenser_checkpoint, None, None)
colCondenser_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  colCondenser_factory.text_scorer()
colCondenser_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  colCondenser_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 17:41:05] #> Loading model checkpoint.
[Dec 01, 17:41:05] #> Loading checkpoint /content/drive/MyDrive/colCondenser-50000.dnn
[Dec 01, 17:41:39] #> checkpoint['epoch'] = 0
[Dec 01, 17:41:39] #> checkpoint['batch'] = 50000


### RD3: Multi-Task Learning (MTL)   

In [23]:
mtl_checkpoint="/content/drive/MyDrive/mtl_v2_cls+tok_losses-50000.dnn" 
mtl_factory = ColBERTFactory(mtl_checkpoint, None, None)
mtl_pipeline_ann = pt.transformer.SourceTransformer(colbert_2019) >>  mtl_factory.text_scorer()
mtl_pipeline = pt.transformer.SourceTransformer(dph_2019) >>  mtl_factory.text_scorer()

/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:233: UserWarning: No index_root and index_name specified - no index ranking possible
  warn("No index_root and index_name specified - no index ranking possible")
/usr/local/lib/python3.7/dist-packages/pyterrier_colbert/ranking.py:249: UserWarning: Faiss not installed. You cannot do retrieval
  warn("Faiss not installed. You cannot do retrieval")
Some weights of the model checkpoint at /content/condenser/ were not used when initializing ColBERT: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids']
- This IS expected if you are initializing ColBERT from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification mo

[Dec 01, 17:41:44] #> Loading model checkpoint.
[Dec 01, 17:41:44] #> Loading checkpoint /content/drive/MyDrive/mtl_v2_cls+tok_losses-50000.dnn
[Dec 01, 17:42:21] #> checkpoint['epoch'] = 0
[Dec 01, 17:42:21] #> checkpoint['batch'] = 50000


## **Experiment**

## DPH(Sparse Retrieval) Re-ranking

In [ ]:
pt.Experiment(
    [colbert_pipeline, cls_similarity_pipeline, token_overlap_pipeline, colCondenser_pipeline, coltacl_pipeline, mtl_pipeline],
    pt.get_dataset('msmarco_passage').get_topics('test-2019'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2019'),
    filter_by_qrels=True,
    names=['DPH >> ColBERT', 'DPH >> ColBERT-increase-cls-similarity', 'DPH >> ColBERT-add-token-overlap', 'DPH >> ColCondenser', 'DPH >> ColTaCL', 'DPH >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

## ColBERT(Dense Retrieval) Re-ranking

In [ ]:
pt.Experiment(
    [colbert_pipeline_ann, cls_similarity_pipeline_ann, token_overlap_pipeline_ann, colCondenser_pipeline_ann, coltacl_pipeline_ann, mtl_pipeline_ann],
    pt.get_dataset('msmarco_passage').get_topics('test-2019'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2019'),
    filter_by_qrels=True,
    names=['ANN-ColBERT >> ColBERT', 'ANN-ColBERT >> ColBERT-increase-cls-similarity', 'ANN-ColBERT >> ColBERT-add-token-overlap', 'ANN-ColBERT >> ColCondenser', 'ANN-ColBERT >> ColTaCL', 'ANN-ColBERT >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

# **TREC 2020 Experiments**

## **Loading model checkpoints**

### Baseline 

In [ ]:
colbert_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  colbert_factory.text_scorer()
colbert_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  colbert_factory.text_scorer()

### RD1: Additional signal during ColBERT training

In [ ]:
cls_similarity_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  cls_similarity_factory.text_scorer()
cls_similarity_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  cls_similarity_factory.text_scorer()

In [ ]:
token_overlap_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  token_overlap_factory.text_scorer()
token_overlap_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  token_overlap_factory.text_scorer()

### RD2: Changing Underlying Pre-trained Model 

In [ ]:
coltacl_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  coltacl_factory.text_scorer()
coltacl_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  coltacl_factory.text_scorer()

In [ ]:
colCondenser_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  colCondenser_factory.text_scorer()
colCondenser_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  colCondenser_factory.text_scorer()

### RD3: Multi-Task Learning (MTL)   

In [ ]:
mtl_pipeline_ann_2020 = pt.transformer.SourceTransformer(colbert_2020) >>  mtl_factory.text_scorer()
mtl_pipeline_2020 = pt.transformer.SourceTransformer(dph_2020) >>  mtl_factory.text_scorer()

# **Experiment**

## DPH(Sparse Retrieval) Re-ranking

In [ ]:
pt.Experiment(
    [colbert_pipeline_2020, cls_similarity_pipeline_2020, token_overlap_pipeline_2020, colCondenser_pipeline_2020, coltacl_pipeline, mtl_pipeline_2020],
    pt.get_dataset('msmarco_passage').get_topics('test-2020'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2020'),
    filter_by_qrels=True,
    names=['DPH >> ColBERT', 'DPH >> ColBERT-increase-cls-similarity', 'DPH >> ColBERT-add-token-overlap', 'DPH >> ColCondenser', 'DPH >> ColTaCL', 'DPH >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)

## ColBERT(Dense Retrieval) Re-ranking

In [ ]:
pt.Experiment(
    [colbert_pipeline_ann_2020, cls_similarity_pipeline_ann_2020, token_overlap_pipeline_ann_2020, colCondenser_pipeline_ann_2020, coltacl_pipeline_ann_2020, mtl_pipeline_ann_2020],
    pt.get_dataset('msmarco_passage').get_topics('test-2020'),
    pt.get_dataset('msmarco_passage').get_qrels('test-2020'),
    filter_by_qrels=True,
    names=['ANN-ColBERT >> ColBERT', 'ANN-ColBERT >> ColBERT-increase-cls-similarity', 'ANN-ColBERT >> ColBERT-add-token-overlap', 'ANN-ColBERT >> ColCondenser', 'ANN-ColBERT >> ColTaCL', 'ANN-ColBERT >> ColBERT-mtl-objective'],
    eval_metrics= [nDCG@10, AP(rel=2)@100, RR(rel=2)@10]
)